## Install Packages

In [1]:
!conda install -y -c conda-forge faiss-gpu
!apt-get -y update
!apt-get -y install libatlas-base-dev

done
Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - faiss-gpu


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.14.0               |   py37h89c1867_0        1010 KB  conda-forge
    toolz-0.12.1               |     pyhd8ed1ab_0          51 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.0 MB

The following NEW packages will be INSTALLED:

  toolz              conda-forge/noarch::toolz-0.12.1-pyhd8ed1ab_0

The following packages will be UPDATED:

  conda                               4.12.0-py37h89c1867_0 --> 4.14.0-py37h89c1867_0



conda-4.14.0         | 1010 KB   | ##################################### | 100% 
toolz-0.12.1         | 51 KB     | ##################################### | 100% 
Preparing transaction: done
Ver

## Load Data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.random_projection import GaussianRandomProjection

from tqdm import tqdm

import faiss

In [3]:
df = pd.read_csv("story_dataset.csv")
df

,prompt_id,prompt,story,hidden_state_file,len_generated_story,len_new_story
0,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Blaz...,./hidden_states/prompt_1.npz,270,271
1,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Spar...,./hidden_states/prompt_1.npz,349,350
2,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Scor...,./hidden_states/prompt_1.npz,278,278
3,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,117,118
4,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,129,130
...,...,...,...,...,...,...
9995,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,289,290
9996,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,119,119
9997,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,127,128
9998,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,441,441


In [4]:
max_story_len = max(df["len_generated_story"])
max_story_len

522

In [5]:
hidden_states_by_layer = {}
NUM_PROMPTS = 10

for prompt_id in range(1, NUM_PROMPTS + 1):
    with np.load(f'./hidden_states/prompt_{prompt_id}.npz') as loaded_data:
        for i in tqdm(range(1000)):
            curr_hidden_states = loaded_data[f"arr_{i}"][0]
#             print(curr_hidden_states.shape)
            for layer in range(4, 5):
                padded_arr = np.zeros((max_story_len, 512))
                padded_arr_len = len(curr_hidden_states[layer][0])
                
                padded_arr[:padded_arr_len] = curr_hidden_states[layer][0]
                
                padded_arr = padded_arr.flatten().astype('float32') #FAISS expects data in type float32 instead of float64 - saves memory too!
#                 print(padded_arr.shape)
                
                if(f"layer_{layer}" in hidden_states_by_layer):
                    hidden_states_by_layer[f"layer_{layer}"].append(padded_arr)
                else:
                    hidden_states_by_layer[f"layer_{layer}"] = [padded_arr]

100%|███████████████████████████████████████████████████████████████████████████████| 1000/1000 [03:06<00:00,  5.36it/s]


In [6]:
layer_hs_array = np.array(hidden_states_by_layer["layer_4"])
layer_hs_array.shape

(10000, 267264)

## Layer 4 Clustering

In [7]:
# Use original vectors for clustering - uncomment next line and comment out last two lines

dim_reduced_vecs = layer_hs_array

# random_projector = GaussianRandomProjection(random_state = 42)
# dim_reduced_vecs = random_projector.fit_transform(layer_hs_array).astype('float32')

In [8]:
dim_reduced_vecs = np.array([v / np.linalg.norm(v) for v in dim_reduced_vecs])
dim_reduced_vecs.shape

(10000, 267264)

In [9]:
# K-means Clustering

ncentroids = NUM_PROMPTS
niter = 20
verbose = True
dim = dim_reduced_vecs.shape[1]
kmeans = faiss.Kmeans(dim, ncentroids, niter = niter, verbose = verbose, gpu = True, nredo = 10, spherical = True, max_points_per_centroid = 1000)
kmeans.train(dim_reduced_vecs)

Clustering 10000 points in 267264D to 10 clusters, redo 10 times, 20 iterations
  Preprocessing in 1.39 s
Outer iteration 0 / 10
  Iteration 19 (25.46 s, search 17.89 s): objective=4820.38 imbalance=1.266 nsplit=0       
Objective improved: keep new clusters
Outer iteration 1 / 10
  Iteration 19 (50.93 s, search 35.72 s): objective=4830.27 imbalance=1.522 nsplit=0       
Objective improved: keep new clusters
Outer iteration 2 / 10
  Iteration 19 (76.42 s, search 53.63 s): objective=4841.97 imbalance=1.158 nsplit=0       
Objective improved: keep new clusters
Outer iteration 3 / 10
  Iteration 19 (101.89 s, search 71.53 s): objective=4832.24 imbalance=1.052 nsplit=0       
Outer iteration 4 / 10
  Iteration 19 (127.25 s, search 89.34 s): objective=4828.22 imbalance=1.117 nsplit=0       
Outer iteration 5 / 10
  Iteration 19 (152.69 s, search 107.21 s): objective=4818.35 imbalance=1.418 nsplit=0       
Outer iteration 6 / 10
  Iteration 19 (178.06 s, search 124.99 s): objective=4824.26 i

4841.97412109375

In [10]:
kmeans.centroids #cluster centers

array([[ 3.5922939e-03,  7.4228978e-05,  5.8439705e-03, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 3.3412278e-03,  6.9038724e-05,  5.4355389e-03, ...,
        -3.3735672e-05,  1.1638801e-05,  1.8126344e-05],
       [ 4.0215938e-03,  8.3096362e-05,  6.5423609e-03, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       ...,
       [ 3.5523197e-03,  7.3401810e-05,  5.7789371e-03, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 4.0774969e-03,  8.4251355e-05,  6.6332920e-03, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 3.9182995e-03,  8.0961960e-05,  6.3743065e-03, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32)

In [11]:
for centroid in kmeans.centroids:
    print(np.linalg.norm(centroid))

1.0000298
1.0000285
1.0000098
1.0000098
1.000019
1.0000163
1.0000123
1.0000579
1.0000124
1.0000114


In [12]:
kmeans.obj #inertia at each iteration

array([2878.77368164, 4654.33203125, 4740.69873047, 4781.28076172,
       4800.99853516, 4809.91601562, 4813.32714844, 4815.31738281,
       4816.5859375 , 4817.48779297, 4818.22119141, 4818.79589844,
       4819.21826172, 4819.60546875, 4819.85986328, 4819.98779297,
       4820.07470703, 4820.16308594, 4820.25585938, 4820.37792969,
       3022.22705078, 4634.40234375, 4683.6640625 , 4736.82519531,
       4779.03027344, 4798.31591797, 4810.51416016, 4817.20361328,
       4821.39990234, 4823.23291016, 4824.35058594, 4825.68994141,
       4826.94433594, 4827.98388672, 4828.76416016, 4829.29150391,
       4829.66064453, 4829.91455078, 4830.11328125, 4830.27001953,
       2818.05615234, 4653.64697266, 4750.78173828, 4785.27636719,
       4801.92724609, 4815.14453125, 4823.81787109, 4830.10107422,
       4833.9765625 , 4836.08105469, 4837.54785156, 4838.50439453,
       4839.20507812, 4839.85205078, 4840.56689453, 4841.18017578,
       4841.46435547, 4841.52685547, 4841.65820312, 4841.97412

In [13]:
normalized_vecs = [v / np.linalg.norm(v) for v in dim_reduced_vecs]

In [14]:
cos_similarities = normalized_vecs @ kmeans.centroids.T
classifications = np.argmax(cos_similarities, axis=1)

In [15]:
pd.Series(classifications).value_counts()

4    1723
2    1395
8    1260
0    1220
9    1023
5     945
7     918
1     573
6     525
3     418
dtype: int64

In [16]:
pd.Series(kmeans.index.search(dim_reduced_vecs.astype(np.float32), 1)[1].flatten()).value_counts()

4    1723
2    1395
8    1260
0    1220
9    1023
5     945
7     918
1     573
6     525
3     418
dtype: int64

In [17]:
prompt_ids = df["prompt_id"]
prompt_ids = prompt_ids.to_numpy()
prompt_ids

array([ 1,  1,  1, ..., 10, 10, 10])

In [18]:
# Get most common centroid for each 1000 points (same label)
max_centroid_per_label = [pd.Series(classifications[i * 1000:(i + 1) * 1000]).value_counts().idxmax() for i in range(10)]
max_centroid_per_label

[4, 9, 6, 4, 9, 8, 4, 9, 4, 1]

In [19]:
# Get most common label for each point classified to a centroid (same centroid)
centroid_labels = [np.where(classifications == i)[0] for i in range(10)]
max_label_per_centroid = [pd.Series(prompt_ids[centroid_labels[i]]).value_counts().idxmax() for i in range(10)]
max_label_per_centroid

[9, 10, 4, 10, 4, 5, 3, 3, 6, 5]

In [20]:
max_centroids = [centroid for centroid in max_centroid_per_label for _ in range(1000)]

In [21]:
np.array(max_centroids)

array([4, 4, 4, ..., 1, 1, 1])

In [22]:
label_to_centroid = {idx + 1 : max_centroid_per_label[idx] for idx in range(len(max_centroid_per_label))}

centroid_to_label = {idx : max_label_per_centroid[idx] for idx in range(len(max_label_per_centroid))}

In [23]:
label_to_centroid

{1: 4, 2: 9, 3: 6, 4: 4, 5: 9, 6: 8, 7: 4, 8: 9, 9: 4, 10: 1}

In [24]:
centroid_to_label

{0: 9, 1: 10, 2: 4, 3: 10, 4: 4, 5: 5, 6: 3, 7: 3, 8: 6, 9: 5}

In [25]:
vectorized_map = np.vectorize(centroid_to_label.get)
classifications_to_label = vectorized_map(classifications)

classifications_to_label

array([ 9,  9,  9, ..., 10, 10, 10])

In [26]:
for i in range(10):
    print(f"Prompt {i + 1} Accuracy: ", np.mean(classifications_to_label[i * 1000:(i + 1) * 1000] == (i + 1)))

Prompt 1 Accuracy:  0.0
Prompt 2 Accuracy:  0.0
Prompt 3 Accuracy:  0.879
Prompt 4 Accuracy:  0.599
Prompt 5 Accuracy:  0.747
Prompt 6 Accuracy:  0.29
Prompt 7 Accuracy:  0.0
Prompt 8 Accuracy:  0.0
Prompt 9 Accuracy:  0.214
Prompt 10 Accuracy:  0.979


In [27]:
print(f"Overall Accuracy: ", np.mean(classifications_to_label == prompt_ids))

Overall Accuracy:  0.3708


In [28]:
for i in range(10):
    print(f"Prompt {i + 1} Accuracy: ", np.mean(classifications[i * 1000:(i + 1) * 1000] == max_centroids[i * 1000:(i + 1)*1000]))

Prompt 1 Accuracy:  0.271
Prompt 2 Accuracy:  0.228
Prompt 3 Accuracy:  0.524
Prompt 4 Accuracy:  0.326
Prompt 5 Accuracy:  0.42
Prompt 6 Accuracy:  0.29
Prompt 7 Accuracy:  0.301
Prompt 8 Accuracy:  0.208
Prompt 9 Accuracy:  0.274
Prompt 10 Accuracy:  0.569


In [29]:
print(f"Overall Accuracy: ", np.mean(classifications == max_centroids))

Overall Accuracy:  0.3411


## Measure Conversion Accuracy

In [30]:
label_to_centroid_vectorized_map = np.vectorize(label_to_centroid.get)
centroid_to_label_vectorized_map = np.vectorize(centroid_to_label.get)
prompt_ids_to_centroids = label_to_centroid_vectorized_map(prompt_ids)
max_centroids_to_labels = centroid_to_label_vectorized_map(max_centroids)

In [31]:
np.mean(max_centroids == prompt_ids_to_centroids) #100% - max_centroids is the prompt ids converted to the max centroid corresponding to each label

1.0

In [32]:
np.mean(max_centroids_to_labels == prompt_ids) #<100%

0.5